In [7]:
-- 1) Limpieza
DROP TABLE IF EXISTS fact_ventas;
DROP TABLE IF EXISTS dim_bodega;
DROP TABLE IF EXISTS dim_producto;
DROP TABLE IF EXISTS dim_cliente;
DROP TABLE IF EXISTS dim_fecha;

-- 2) Dimensión de Fechas (Delta)
CREATE TABLE dim_fecha (
  datekey       INT            NOT NULL,   -- YYYYMMDD
  `date`        DATE           NOT NULL,
  `year`        INT            NOT NULL,
  monthnumber   SMALLINT,
  monthname     STRING,
  quarter       STRING
)
USING DELTA;

-- Carga calendario (EJEMPLO: todo 2025). Ajusta el rango si quieres.
INSERT INTO dim_fecha
SELECT
  CAST(date_format(d, 'yyyyMMdd') AS INT) AS datekey,
  d                                      AS `date`,
  YEAR(d)                                AS `year`,
  MONTH(d)                               AS monthnumber,
  date_format(d,'MMMM')                  AS monthname,
  CONCAT('Q', QUARTER(d))                AS quarter
FROM (
  SELECT explode(sequence(to_date('2025-01-01'), to_date('2025-12-31'), interval 1 day)) AS d
);

-- 3) Dimensión de Clientes (Delta)
CREATE TABLE dim_cliente (
  clientekey     BIGINT         NOT NULL,     -- surrogate manejado por tu ETL
  clientenk      STRING         NOT NULL,     -- natural key
  clientenombre  STRING         NOT NULL,
  segmento       STRING,
  email          STRING
)
USING DELTA;

INSERT INTO dim_cliente (clientekey, clientenk, clientenombre, segmento, email) VALUES
(1, 'C001', 'Juan Pérez',   'Retail',  'juan.perez@example.com'),
(2, 'C002', 'Lucía Ramos',  'Premium', 'lucia.ramos@example.com'),
(3, 'C003', 'Comercial XYZ','Empresa', 'contacto@comercialxyz.pe');

-- 4) Dimensión de Productos (Delta)
CREATE TABLE dim_producto (
  productokey     BIGINT         NOT NULL,
  productonk      STRING         NOT NULL,
  productonombre  STRING         NOT NULL,
  categoria       STRING,
  subcategoria    STRING,
  preciolista     DECIMAL(18,2)
)
USING DELTA;

INSERT INTO dim_producto (productokey, productonk, productonombre, categoria, subcategoria, preciolista) VALUES
(1, 'P001', 'Laptop Pro 14',      'Electrónica', 'Computo',   5200.00),
(2, 'P002', 'Smartphone Max 6.5', 'Electrónica', 'Teléfonos', 2800.00),
(3, 'P003', 'Auriculares ANC',    'Accesorios',  'Audio',      650.00);

-- 5) Dimensión de Bodegas (Delta)
CREATE TABLE dim_bodega (
  bodegakey      BIGINT         NOT NULL,
  bodegank       STRING         NOT NULL,
  bodeganombre   STRING         NOT NULL,
  region         STRING,
  ciudad         STRING
)
USING DELTA;

INSERT INTO dim_bodega (bodegakey, bodegank, bodeganombre, region, ciudad) VALUES
(1, 'B001', 'Lima Centro',  'Costa',  'Lima'),
(2, 'B002', 'Huancayo HQ',  'Sierra', 'Huancayo'),
(3, 'B003', 'Arequipa Sur', 'Sur',    'Arequipa');

-- 6) Tabla de Hechos (Delta)
CREATE TABLE fact_ventas (
  factid         BIGINT         NOT NULL,
  datekey        INT            NOT NULL,   -- referencia a dim_fecha.datekey
  clientekey     BIGINT         NOT NULL,   -- referencia a dim_cliente.clientekey
  productokey    BIGINT         NOT NULL,   -- referencia a dim_producto.productokey
  bodegakey      BIGINT         NOT NULL,   -- referencia a dim_bodega.bodegakey
  idtransaccion  STRING,
  cantidad       INT,
  importe        DECIMAL(18,2),
  descuento      DECIMAL(18,2),
  costo          DECIMAL(18,2)
)
USING DELTA;

-- 7) Carga de ejemplo (Hechos)
INSERT INTO fact_ventas (factid, datekey, clientekey, productokey, bodegakey, idtransaccion, cantidad, importe, descuento, costo)
VALUES
(1, 20250115, 1, 1, 1, 'TXN-0001', 1, 5200.00,   0.00, 4300.00),
(2, 20250510, 2, 3, 2, 'TXN-0002', 2, 5600.00, 100.00, 4400.00),
(3, 20251003, 3, 2, 3, 'TXN-0003', 1, 2800.00,   0.00, 2300.00);

-- 8) Verificación rápida
DESCRIBE EXTENDED dim_fecha;
DESCRIBE EXTENDED fact_ventas;

SELECT COUNT(*) AS cnt_dim_fecha FROM dim_fecha;
SELECT * FROM fact_ventas;

StatementMeta(, 774aa65c-96d8-4472-97db-06867d7cddbb, 66, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

...

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 14 rows and 3 fields>

<Spark SQL result set with 18 rows and 3 fields>

<Spark SQL result set with 1 rows and 1 fields>

<Spark SQL result set with 3 rows and 10 fields>